In [ ]:
import time
import shioaji as sj
import pandas as pd
from datetime import date
import json
import requests
from google.oauth2.service_account import Credentials
import gspread
#from bucket import connect_storage

#connect to storage
#connect_storage()

# Login
api = sj.Shioaji(simulation=False) ###True為測試環境###
api.login("#API TOKEN",
        "#API TOKEN")

print('Login')

auth_json_path = 'pythonsheetapi-384805-70a2e2836d4f.json'
scope = ['https://www.googleapis.com/auth/spreadsheets']

creds = Credentials.from_service_account_file(auth_json_path, scopes=scope)
gs = gspread.authorize(creds)

sheet = gs.open_by_url('https://docs.google.com/spreadsheets/d/1cOPhYayW8VHEkvGLqqveeL8mozyIzCh8x6U7jQB7Y_Q/edit#gid=0')
worksheet = sheet.get_worksheet(0)

positions = api.list_positions(api.futopt_account)
df = pd.DataFrame(p.__dict__ for p in positions)
df.drop(columns='id',inplace=True)
df['Date'] = date.today().strftime('%Y-%m-%d')
values = df.values.tolist()

def update(values):
    # Check row
    list_of_lists = worksheet.get_all_values()
    row_n = len(list_of_lists)

    # Update the worksheet with the values
    for sublist in values:
        worksheet.insert_row(sublist, row_n + 1)  # Insert each value as a new row
    
    return 'Saved!'

# Call the 'update' function with the values from the DataFrame
result = update(values)

In [ ]:
positions = api.list_positions(api.futopt_account)
df = pd.DataFrame(p.__dict__ for p in positions)

In [ ]:
def dailypnl():
    #connect to storage
    connect_storage()

    # Login
    api = sj.Shioaji(simulation=False) ###True為測試環境###
    api.login("#API TOKEN",
            "#API TOKEN")

    print('Login')

    # Authorise
    auth_json_path = 'tmp/pythonsheetapi-384805-70a2e2836d4f.json'
    scope = ['https://www.googleapis.com/auth/spreadsheets']

    creds = Credentials.from_service_account_file(auth_json_path, scopes=scope)
    gs = gspread.authorize(creds)

    # Connect to worksheet
    sheet = gs.open_by_url('https://docs.google.com/spreadsheets/d/1cOPhYayW8VHEkvGLqqveeL8mozyIzCh8x6U7jQB7Y_Q/edit#gid=0')
    worksheet = sheet.get_worksheet(0)

    # Get Data
    positions = api.list_positions(api.futopt_account)
    df = pd.DataFrame(p.__dict__ for p in positions)
    df.drop(columns='id',inplace=True)
    df['Date'] = date.today().strftime('%Y-%m-%d')
    values = df.values.tolist()

    # Get a flattened list of values from the DataFrame
    values = [val for sublist in df.values.tolist() for val in sublist]

    def update(values):
        # Check row
        list_of_lists = worksheet.get_all_values()
        row_n = len(list_of_lists)

        # Update at last row
        worksheet.insert_row(values, row_n+1) #插入values到第1列
        
        return 'Saved!'
    
    # Line notify
    def lineNotifyMessage(msg):

            headers = {
            "Authorization": "Bearer " + '#API TOKEN', 
            "Content-Type" : "application/x-www-form-urlencoded"
            }

            payload = {'message': msg }
            r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)

    lineNotifyMessage("{} Data Inserted".format(df))

    # Call the 'update' function with the values from the DataFrame
    update(values)
    return "Data Upadted"

    